<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#sqlite3-method" data-toc-modified-id="sqlite3-method-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>sqlite3 method</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine, MetaData, Table, inspect, insert
from sqlalchemy.orm import sessionmaker

# sqlite3 method

In [2]:
import sys
sys.path.append('../PEAS/app')
sys.path.append('../PEAS/db_build')
import db_factory

In [3]:
mars_spectrum = pd.read_csv('../PEAS/app/data/mars_spectrum.dat', delim_whitespace=True, header=None, names=['wavelength','flux'])

In [ ]:
#data 1
meta1 = {
    'UT_date':'2021-08-10',
    'time_of_obs':'10:05:35',
    'object_name':'Mars',
    'integration_time':3.89,
    'grating':2,
    'central_wavelength':640.2,
    'slit_width':300,
    'phase_angle':45.3,
    'comments':'This is a test'
}
wave1 =  mars_spectrum['wavelength'].values
flux1 = mars_spectrum['flux'].values

#data 2
meta2 = {
    'UT_date':'2021-08-12',
    'time_of_obs':'11:45:22',
    'object_name':'Jupiter',
    'integration_time':3.89,
    'grating':2,
    'central_wavelength':640.2,
    'slit_width':300,
    'phase_angle':45.3,
    'comments':'This is test 2'
}
wave2 = np.array([7,3,8,2])
flux2 = np.array([5,4,5,1])


In [ ]:
meta1['UT_date']

In [ ]:
sqlite3_db = db_factory.sqlite3_db('sqlite3.db')
#create empty database
#sqlite3_db.create_db()

In [ ]:
#insert data
sqlite3_db.sqlite3_insert(meta1, wave1, spectrum1)

In [ ]:
#read data
sqlite3_db.sqlite3_query('Mars')

In [ ]:
%timeit sqlite3_db.sqlite3_query('Mars')

In [ ]:
import os


def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [ ]:
file_size('sqlite3.db')

# SQLAlchemy method

In [ ]:
#create empty database
db_factory.sqlalchemy_db.create_db()

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
result = engine.execute('PRAGMA table_info(meta);')
for i in result:
    print(i)

In [ ]:
db_factory.grab_sqlalchemy.alchemy_query(2)

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
result = engine.execute("""INSERT INTO 'meta'(UT_date, time_of_obs, object_name, integration_time, grating, central_wavelength, slit_width, phase_angle, comments)
                        VALUES ("2021-08-19", "10:22:23", "Mars", 3.89, 2, 640.2, 300, 45.3, "This is a test");""")

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')

result = engine.execute(f"""INSERT INTO 'meta'(UT_date, time_of_obs, object_name, integration_time, grating, central_wavelength, slit_width, phase_angle, comments)
                     VALUES ("{meta1['UT_date']}", "{meta1['time_of_obs']}", "{meta1['object_name']}", {meta1['integration_time']}, {meta1['grating']}, {meta1['central_wavelength']}, {meta1['slit_width']}, {meta1['phase_angle']},"{meta1['comments']}");""")


In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
result = engine.execute('SELECT * FROM meta;')
for i in result:
    print(i)
    
result.lastrowid

In [ ]:
db_factory.grab_sqlalchemy.alchemy_insert(2, meta2)

In [ ]:
db_factory.grab_sqlalchemy.alchemy_query(2)

In [ ]:
from sqlalchemy import MetaData 
meta = MetaData(engine).reflect()
user_t = meta.tables['meta']

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
inspector = inspect(engine)
print(inspector.get_table_names())

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
result = engine.execute('SELECT * FROM meta;')
for i in result:
    print(i)

In [ ]:
# Create connection
conn = engine.connect()
# Begin transaction
trans = conn.begin()
result = conn.execute(f"""INSERT INTO 'meta'(UT_date, time_of_obs, object_name, integration_time, grating, central_wavelength, slit_width, phase_angle, comments)
                     VALUES ("{meta1['UT_date']}", "{meta1['time_of_obs']}", "{meta1['object_name']}", {meta1['integration_time']}, {meta1['grating']}, {meta1['central_wavelength']}, {meta1['slit_width']}, {meta1['phase_angle']},"{meta1['comments']}");""")
trans.commit()
print(result.lastrowid)
# Close connection
conn.close()


In [ ]:
conn.close()

In [ ]:
db_path = '/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3'
engine = create_engine(f'sqlite:///{db_path}')
conn = engine.connect()
trans = conn.begin()
result = engine.execute(f"""INSERT INTO 'meta'(UT_date, time_of_obs, object_name, integration_time, grating, central_wavelength, slit_width, phase_angle, comments)
                     VALUES ("{meta2['UT_date']}", "{meta2['time_of_obs']}", "{meta2['object_name']}", {meta2['integration_time']}, {meta1['grating']}, {meta1['central_wavelength']}, {meta1['slit_width']}, {meta1['phase_angle']},"{meta1['comments']}");""")
ID = result.lastrowid
for x, y in zip(wave2, flux2):
    print(x,y)
    result = engine.execute(f"""INSERT INTO 'spectra'(wave, flux, meta_id)
                                VALUES ({x}, {y}, {ID});""")
trans.commit()
conn.close()

In [ ]:
for x, y in zip(wave2, spectrum2):
    print(x, y)

In [ ]:
db_factory.grab_sqlalchemy.alchemy_insert(2, meta1, wave1, flux1)

In [54]:
%timeit db_factory.grab_sqlalchemy.alchemy_query('Mars')

(1, '2021-08-10', '10:05:35', 'Mars', 3.89, 2, 640.2, 300, 45.3, 'This is a test')
(1, 450.6108737759, 3023.780918937958, 1)
(2, 451.124956064013, 3840.8019209405256, 1)
(3, 451.6390345728768, 2860.418989153304, 1)
(4, 452.1531092925677, 3854.5787284703006, 1)
(5, 452.6671802131624, 3073.0606745178407, 1)
(6, 453.1812473247375, 3840.683505849068, 1)
(7, 453.6953106173696, 3068.118757323035, 1)
(8, 454.20937008113526, 3836.934836634236, 1)
(9, 454.7234257061112, 3057.7229262602177, 1)
(10, 455.23747748237383, 3822.7782481292015, 1)
(11, 455.7515254, 3039.661635019464, 1)
(12, 456.2655694490661, 3797.680441507652, 1)
(13, 456.7796096196488, 3046.559128335992, 1)
(14, 457.29364590182473, 3873.241975723407, 1)
(15, 457.8076782856704, 3298.4089776216215, 1)
(16, 458.3217067612625, 3771.469263974876, 1)
(17, 458.8357313186776, 3438.22971581965, 1)
(18, 459.34975194799233, 4037.4900433357343, 1)
(19, 459.8637686392832, 3074.3316128046977, 1)
(20, 460.3777813826269, 3583.6865872110775, 1)
(21,

(64, 482.99028298480965, 3525.167227976123, 1)
(65, 483.5041077998104, 3193.946243277755, 1)
(66, 484.0179282103874, 3672.0775076560785, 1)
(67, 484.5317442066176, 3075.9648898276014, 1)
(68, 485.0455557785773, 3628.2291681932734, 1)
(69, 485.5593629163432, 3206.9450038509244, 1)
(70, 486.0731656099919, 3726.107770450712, 1)
(71, 486.5869638496, 3435.8651586120814, 1)
(72, 487.10075762524406, 3926.172819020329, 1)
(73, 487.6145469270008, 3334.200627385447, 1)
(74, 488.1283317449467, 3769.378468088321, 1)
(75, 488.6421120691584, 3403.672002142035, 1)
(76, 489.15588788971246, 3740.346197241767, 1)
(77, 489.6696591966856, 3415.202159944228, 1)
(78, 490.1834259801543, 3866.8130373033514, 1)
(79, 490.6971882301952, 3362.1414757140715, 1)
(80, 491.21094593688485, 3770.0503107041295, 1)
(81, 491.7246990903, 3460.0056249196027, 1)
(82, 492.238447680517, 4029.403471823052, 1)
(83, 492.7521916976128, 3348.0279316311758, 1)
(84, 493.2659311316637, 4081.8522126988946, 1)
(85, 493.7796659727464, 35

(42, 471.68504282058507, 3808.337531165817, 1)
(43, 472.1989620222848, 3106.264068388397, 1)
(44, 472.71287703787567, 3755.3658848467376, 1)
(45, 473.2267878574344, 3258.3957023909, 1)
(46, 473.7406944710375, 3909.0945816362614, 1)
(47, 474.2545968687616, 2902.1783686218128, 1)
(48, 474.76849504068326, 3513.51507791536, 1)
(49, 475.2823889768792, 3278.302628912452, 1)
(50, 475.79627866742584, 3870.189340702482, 1)
(51, 476.3101641024, 3303.379463047229, 1)
(52, 476.8240452718781, 3797.376688467656, 1)
(53, 477.3379221659368, 3229.1624847250687, 1)
(54, 477.8517947746527, 3783.0958021870206, 1)
(55, 478.3656630881024, 3303.599392705245, 1)
(56, 478.8795270963625, 3817.208313289266, 1)
(57, 479.3933867895096, 3410.7571812444125, 1)
(58, 479.9072421576203, 3963.901250814896, 1)
(59, 480.4210931907712, 3218.612200615708, 1)
(60, 480.9349398790389, 3790.093916285255, 1)
(61, 481.4487822125, 3376.7926716802426, 1)
(62, 481.9626201812311, 3976.4439281898817, 1)
(63, 482.4764537753088, 3185.96

(512, 712.5950772632161, 9639.421633195834, 1)
(513, 713.1059352862487, 9347.386102991108, 1)
(514, 713.6167844591746, 9523.596604735676, 1)
(515, 714.1276247720705, 9346.243506435396, 1)
(516, 714.6384562150125, 9651.3221937866, 1)
(517, 715.1492787780776, 9235.938317315273, 1)
(518, 715.6600924513423, 9351.416663919272, 1)
(519, 716.1708972248832, 8983.891250941437, 1)
(520, 716.6816930887769, 9010.83761179832, 1)
(521, 717.1924800331001, 8645.792066911956, 1)
(522, 717.703258047929, 9147.775999235952, 1)
(523, 718.2140271233408, 8735.436000496096, 1)
(524, 718.7247872494116, 8951.540742689707, 1)
(525, 719.2355384162183, 8683.353257678735, 1)
(526, 719.7462806138376, 9356.926183422525, 1)
(527, 720.2570138323456, 9006.401988736385, 1)
(528, 720.7677380618192, 9322.121235220699, 1)
(529, 721.278453292335, 8820.419763995358, 1)
(530, 721.7891595139699, 9054.100164812327, 1)
(531, 722.2998567167998, 8819.843008940945, 1)
(532, 722.8105448909021, 9038.849673990113, 1)
(533, 723.32122402

(300, 604.1093409397508, 8026.521115483034, 1)
(301, 604.6218511349, 7684.77350481231, 1)
(302, 605.134354583703, 8186.054435776978, 1)
(303, 605.6468512762368, 7777.752156175761, 1)
(304, 606.1593412025777, 8072.477185660335, 1)
(305, 606.6718243528023, 7970.975238998312, 1)
(306, 607.1843007169875, 8101.762847418116, 1)
(307, 607.6967702852096, 7724.383481221098, 1)
(308, 608.2092330475452, 7919.001375282403, 1)
(309, 608.7216889940712, 7640.245991423395, 1)
(310, 609.2341381148639, 8058.400751707254, 1)
(311, 609.7465804000001, 7724.556032282045, 1)
(312, 610.259015839556, 7983.338860074221, 1)
(313, 610.7714444236087, 7693.0016552836305, 1)
(314, 611.2838661422347, 8087.170975539015, 1)
(315, 611.7962809855103, 7739.057199447835, 1)
(316, 612.3086889435125, 8248.681128724022, 1)
(317, 612.8210900063176, 7881.149955875952, 1)
(318, 613.3334841640022, 8073.596832841771, 1)
(319, 613.8458714066433, 7893.063486243343, 1)
(320, 614.3582517243169, 8102.476401409402, 1)
(321, 614.87062510

...

(465, 688.5749500889303, 8386.581356884051, 1)
(466, 689.0862128733875, 8998.976381434035, 1)
(467, 689.5974672741377, 8812.958624696208, 1)
(468, 690.1087132812572, 9309.873624254189, 1)
(469, 690.6199508848231, 9030.221960355404, 1)
(470, 691.1311800749119, 9377.173756686461, 1)
(471, 691.6424008416, 9057.02168184504, 1)
(472, 692.153613174964, 9509.780335946887, 1)
(473, 692.6648170650808, 9101.247028797396, 1)
(474, 693.1760125020267, 9599.566377483308, 1)
(475, 693.6871994758784, 9126.248832981788, 1)
(476, 694.1983779767124, 9607.870485763624, 1)
(477, 694.7095479946056, 9388.850019930815, 1)
(478, 695.2207095196343, 9459.145377819812, 1)
(479, 695.7318625418752, 9359.33394257328, 1)
(480, 696.2430070514049, 9695.939221969187, 1)
(481, 696.7541430383001, 9451.21486885125, 1)
(482, 697.265270492637, 9581.596822156835, 1)
(483, 697.7763894044929, 9437.459475228146, 1)
(484, 698.2874997639436, 9684.428399929247, 1)
(485, 698.7986015610663, 9370.783759111537, 1)
(486, 699.30969478593

(189, 547.1810974166472, 5242.371068637929, 1)
(190, 547.6942947723479, 5911.515647757567, 1)
(191, 548.2074864832, 5420.17280147145, 1)
(192, 548.72067253928, 5707.023559949765, 1)
(193, 549.2338529306649, 5140.4407079880575, 1)
(194, 549.7470276474307, 5807.4938600278465, 1)
(195, 550.2601966796544, 5274.376006768512, 1)
(196, 550.7733600174124, 5853.5046109066525, 1)
(197, 551.2865176507817, 5410.38528504421, 1)
(198, 551.7996695698382, 5866.768715465016, 1)
(199, 552.3128157646591, 5496.494881104415, 1)
(200, 552.8259562253209, 5996.37094600587, 1)
(201, 553.3390909418999, 5644.783416767742, 1)
(202, 553.8522199044729, 5896.375628243574, 1)
(203, 554.3653431031169, 5543.085777419554, 1)
(204, 554.8784605279077, 5784.88291886539, 1)
(205, 555.3915721689224, 5583.929479127178, 1)
(206, 555.9046780162375, 6010.687029071579, 1)
(207, 556.4177780599296, 5525.611951562992, 1)
(208, 556.9308722900753, 5970.096732692747, 1)
(209, 557.4439606967512, 5710.929795549845, 1)
(210, 557.957043270

(630, 772.8125829795998, 8433.764909524707, 1)
(631, 773.3223281887998, 7953.164790977363, 1)
(632, 773.832063376932, 8208.28737178434, 1)
(633, 774.3417885340727, 7982.484538932737, 1)
(634, 774.8515036502986, 8314.517312678749, 1)
(635, 775.3612087156864, 8066.666423121297, 1)
(636, 775.8709037203125, 8195.408987280545, 1)
(637, 776.3805886542536, 7986.515876425715, 1)
(638, 776.8902635075863, 8263.201298644217, 1)
(639, 777.399928270387, 7891.010624255641, 1)
(640, 777.9095829327327, 8282.357888478202, 1)
(641, 778.4192274847, 7884.295477673134, 1)
(642, 778.9288619163651, 8216.334772838614, 1)
(643, 779.438486217805, 7926.090340835357, 1)
(644, 779.9481003790956, 8075.212697571006, 1)
(645, 780.4577043903142, 7947.990568413584, 1)
(646, 780.9672982415376, 8354.857248649976, 1)
(647, 781.4768819228416, 7917.851801133231, 1)
(648, 781.9864554243034, 8158.814431048429, 1)
(649, 782.4960187359992, 7717.431561, 1)
(650, 783.0055718480057, 8292.45120432191, 1)
(651, 783.5151147504, 7817.

(26, 463.46177438008743, 3713.3803020284186, 1)
(27, 463.9757592099456, 3084.023434672443, 1)
(28, 464.48974001246927, 3817.140318323825, 1)
(29, 465.0037167777352, 3112.8607801385815, 1)
(30, 465.5176894958199, 3888.269404720699, 1)
(31, 466.0316581568, 3205.648082355899, 1)
(32, 466.5456227507521, 3940.378375009625, 1)
(33, 467.0595832677528, 3130.772808598538, 1)
(34, 467.57353969787863, 3742.251885681376, 1)
(35, 468.0874920312064, 3257.100817718628, 1)
(36, 468.6014402578125, 3623.6461774809286, 1)
(37, 469.1153843677736, 3269.0111807266535, 1)
(38, 469.6293243511663, 3651.8541830143217, 1)
(39, 470.1432601980672, 3155.3894931095556, 1)
(40, 470.65719189855287, 3860.5180480264767, 1)
(41, 471.1711194427, 2937.200952216663, 1)
(42, 471.68504282058507, 3808.337531165817, 1)
(43, 472.1989620222848, 3106.264068388397, 1)
(44, 472.71287703787567, 3755.3658848467376, 1)
(45, 473.2267878574344, 3258.3957023909, 1)
(46, 473.7406944710375, 3909.0945816362614, 1)
(47, 474.2545968687616, 290

(447, 679.3707976674816, 9310.372300303912, 1)
(448, 679.8822096617633, 9791.4318906944, 1)
(449, 680.3936134509593, 9476.666978831336, 1)
(450, 680.9050090251458, 9693.417725941594, 1)
(451, 681.4163963743999, 9421.86877094763, 1)
(452, 681.927775488798, 9681.839534237552, 1)
(453, 682.4391463584168, 9399.427269071566, 1)
(454, 682.9505089733327, 9558.780806917108, 1)
(455, 683.4618633236224, 9044.826796749896, 1)
(456, 683.9732093993625, 9255.306717899351, 1)
(457, 684.4845471906297, 8914.727248877116, 1)
(458, 684.9958766875002, 8642.335406248569, 1)
(459, 685.5071978800511, 7931.2651488624015, 1)
(460, 686.0185107583588, 8250.588493301293, 1)
(461, 686.5298153125, 8036.113426121731, 1)
(462, 687.0411115325511, 8424.119948769821, 1)
(463, 687.5523994085888, 8284.337707362258, 1)
(464, 688.0636789306898, 8563.906740170245, 1)
(465, 688.5749500889303, 8386.581356884051, 1)
(466, 689.0862128733875, 8998.976381434035, 1)
(467, 689.5974672741377, 8812.958624696208, 1)
(468, 690.108713281

In [55]:
file_size('/Users/Doris/Documents/codes/peas/PEAS/db_build/alchemy_db.sqlite3')

NameError: name 'file_size' is not defined

In [4]:
mars_spectrum

,wavelength,flux
0,450.610874,3023.780919
1,451.124956,3840.801921
2,451.639035,2860.418989
3,452.153109,3854.578728
4,452.667180,3073.060675
...,...,...
729,823.735895,7612.270107
730,824.244589,7287.790936
731,824.753272,7623.684440
732,825.261944,7369.832744


In [8]:
json_db = mars_spectrum.to_json(indent=2)
print(json_db)

{
  "wavelength":{
    "0":450.6108737759,
    "1":451.124956064,
    "2":451.6390345729,
    "3":452.1531092926,
    "4":452.6671802132,
    "5":453.1812473247,
    "6":453.6953106174,
    "7":454.2093700811,
    "8":454.7234257061,
    "9":455.2374774824,
    "10":455.7515254,
    "11":456.2655694491,
    "12":456.7796096196,
    "13":457.2936459018,
    "14":457.8076782857,
    "15":458.3217067613,
    "16":458.8357313187,
    "17":459.349751948,
    "18":459.8637686393,
    "19":460.3777813826,
    "20":460.8917901681,
    "21":461.4057949858,
    "22":461.9197958257,
    "23":462.4337926781,
    "24":462.9477855328,
    "25":463.4617743801,
    "26":463.9757592099,
    "27":464.4897400125,
    "28":465.0037167777,
    "29":465.5176894958,
    "30":466.0316581568,
    "31":466.5456227508,
    "32":467.0595832678,
    "33":467.5735396979,
    "34":468.0874920312,
    "35":468.6014402578,
    "36":469.1153843678,
    "37":469.6293243512,
    "38":470.1432601981,
    "39":470.65719189

In [6]:
json_index = mars_spectrum.to_json(orient = 'index')
print(json_index, "\n")

{"0":{"wavelength":450.6108737759,"flux":3023.780918938},"1":{"wavelength":451.124956064,"flux":3840.8019209405},"2":{"wavelength":451.6390345729,"flux":2860.4189891533},"3":{"wavelength":452.1531092926,"flux":3854.5787284703},"4":{"wavelength":452.6671802132,"flux":3073.0606745178},"5":{"wavelength":453.1812473247,"flux":3840.6835058491},"6":{"wavelength":453.6953106174,"flux":3068.118757323},"7":{"wavelength":454.2093700811,"flux":3836.9348366342},"8":{"wavelength":454.7234257061,"flux":3057.7229262602},"9":{"wavelength":455.2374774824,"flux":3822.7782481292},"10":{"wavelength":455.7515254,"flux":3039.6616350195},"11":{"wavelength":456.2655694491,"flux":3797.6804415077},"12":{"wavelength":456.7796096196,"flux":3046.559128336},"13":{"wavelength":457.2936459018,"flux":3873.2419757234},"14":{"wavelength":457.8076782857,"flux":3298.4089776216},"15":{"wavelength":458.3217067613,"flux":3771.4692639749},"16":{"wavelength":458.8357313187,"flux":3438.2297158197},"17":{"wavelength":459.3497519